In [16]:
import sys
import pandas as pd
import numpy as np
import random
import requests
import json
import matplotlib.pyplot as plt
import time
from nltk.book import *
from nltk.corpus import stopwords
import nltk.classify.util
from nltk.classify import NaiveBayesClassifier
import random
from nltk import classify
from nltk import NaiveBayesClassifier
from nltk.tokenize import word_tokenize
import seaborn as sns
import pyLDAvis
import pyLDAvis.gensim
sns.set_context('notebook')
import warnings
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer
from gensim import corpora, models
import gensim
from nltk.corpus import stopwords
warnings.filterwarnings('ignore')

*** Introductory Examples for the NLTK Book ***
Loading text1, ..., text9 and sent1, ..., sent9
Type the name of the text or sentence to view it.
Type: 'texts()' or 'sents()' to list the materials.
text1: Moby Dick by Herman Melville 1851
text2: Sense and Sensibility by Jane Austen 1811
text3: The Book of Genesis
text4: Inaugural Address Corpus
text5: Chat Corpus
text6: Monty Python and the Holy Grail
text7: Wall Street Journal
text8: Personals Corpus
text9: The Man Who Was Thursday by G . K . Chesterton 1908


/usr/local/lib/python3.7/site-packages/botocore/awsrequest.py:624: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  class HeadersDict(collections.MutableMapping):
/usr/local/lib/python3.7/site-packages/gensim/corpora/dictionary.py:11: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Mapping, defaultdict
/usr/local/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()


In [17]:
#### get all csv dataframes for trending
US_trending_df = pd.read_csv('youtube-new/USvideos.csv') #USA
CA_trending_df = pd.read_csv('youtube-new/CAvideos.csv') #CANADA
DE_trending_df = pd.read_csv('youtube-new/DEvideos.csv') #GERMANY
FR_trending_df = pd.read_csv('youtube-new/FRvideos.csv') #FRANCE
GB_trending_df = pd.read_csv('youtube-new/GBvideos.csv') #GREAT BRITAIN
IN_trending_df = pd.read_csv('youtube-new/INvideos.csv') #INDIA

# JP_trending_df = pd.read_csv('youtube-new/JPvideos.csv', encoding="UTF-8") #JAPAN

# KR_trending_df = pd.read_csv('youtube-new/KRvideos.csv') #SOUTH KOREA

# MX_trending_df = pd.read_csv('youtube-new/MXvideos.csv') #MEXICO

# RU_trending_df = pd.read_csv('youtube-new/RUvideos.csv') #RUSSIA

list_of_all_trending_dfs = [US_trending_df, CA_trending_df, DE_trending_df, FR_trending_df, GB_trending_df, IN_trending_df]
full_trending_df = pd.concat(list_of_all_trending_dfs)

In [36]:
f = open("apiKey", "r")
key = f.read()
## for each videoId, find a related video
def do_search_youtube_request(videoId):
    url = "https://www.googleapis.com/youtube/v3/search?part=snippet&maxResults=50&order=relevance&relatedToVideoId={}&type=video&videoDefinition=any&key={}".format(videoId, key)
    r = requests.get(url)
    return r

## given a set of videoIds, find insights (statistics, tags, etc)
def find_video_insights(videoIds):
    print(videoIds)
    url = 'https://www.googleapis.com/youtube/v3/videos?part=snippet%2CcontentDetails%2Cstatistics&id={}&key={}'.format(videoIds, key)
    r = requests.get(url)
    return r

## call this with 1 country at a time 
def process_youtube_requests(videoIds):
    df = pd.DataFrame(columns=['video_id', 'title', 'channel_title', 'category_id', 'publish_time', 'tags', 
                               'views', 'likes', 'dislikes', 'comment_count', 'description'])
    relatedVideoIds = []
    for videoId in videoIds:
        try: 
            response = do_search_youtube_request(videoId)
            time.sleep(2)
            if (response.status_code == 200):
                r1 = response.json()
                relatedVideoIdItems = r1['items']
                for id in relatedVideoIdItems:
                    relatedVideoId = id['id']['videoId']
                    relatedVideoIds.append(str(relatedVideoId))
            else:
                print(response.status_code)
        except:
            print ("Something went wrong here! 2")
    random.shuffle(relatedVideoIds)
    videoIdsForInsights = []
    for i in range(0, len(relatedVideoIds), 50):
        videoIdsForInsights.append(relatedVideoIds[i:i + n])
    for videoIdList in videoIdsForInsights:
        videoIdsStr = '%2C'.join([str(elem) for elem in videoIdList])
        r2 = find_video_insights(videoIdsStr)
        time.sleep(2)
        if (r2.status_code == 200):
            r = r2.json()
            i = 0
            while (i < len(videoIdList)):
                try:
                    id = videoIdList[i]
                    title = (r['items'][i]['snippet']['title'])
                    channel_title = (r['items'][i]['snippet']['channelTitle'])
                    category_id = (r['items'][i]['snippet']['categoryId'])
                    publish_time = (r['items'][i]['snippet']['publishedAt'])
                    tags = '|'.join((r['items'][i]['snippet']['tags']))
                    views = (r['items'][i]['statistics']['viewCount'])
                    likes = (r['items'][i]['statistics']['likeCount'])
                    dislikes = (r['items'][i]['statistics']['dislikeCount'])
                    comment_count = (r['items'][i]['statistics']['commentCount'])
                    description = (r['items'][i]['snippet']['description'])
                    data = {'video_id': id, 'title': title, 'channel_title': channel_title, 'category_id' : category_id,
                           'publish_time' : publish_time, 'tags' : tags, 'views' : views, 'likes' : likes, 'dislikes' : dislikes,
                           'comment_count' : comment_count, 'description' : description}
                    df = df.append(data, ignore_index = True)
                except:
                    print("Something went wrong! 3")
                i = i + 1
        else:
            print("Something went wrong! 4")
            print (r2.status_code)
            print(r2.text)
    return df
        

In [19]:
n = 50
US_trending_videoIds = US_trending_df.sample(n)['video_id'].tolist()
CA_trending_videoIds = CA_trending_df.sample(n)['video_id'].tolist()
DE_trending_videoIds = DE_trending_df.sample(n)['video_id'].tolist()
FR_trending_videoIds = FR_trending_df.sample(n)['video_id'].tolist()
GB_trending_videoIds = GB_trending_df.sample(n)['video_id'].tolist()
IN_trending_videoIds = IN_trending_df.sample(n)['video_id'].tolist()

In [ ]:
## do following requests separately with a new API Key
not_trending_us_df = process_youtube_requests(US_trending_videoIds)
not_trending_ca_df = process_youtube_requests(CA_trending_videoIds)
not_trending_de_df = process_youtube_requests(DE_trending_videoIds)
not_trending_fr_df = process_youtube_requests(FR_trending_videoIds)
not_trending_gb_df = process_youtube_requests(GB_trending_videoIds)
not_trending_in_df = process_youtube_requests(IN_trending_videoIds)

In [45]:
list_of_all_nontrending_dfs = [not_trending_us_df, not_trending_ca_df, not_trending_de_df, 
                              not_trending_fr_df, not_trending_gb_df, not_trending_in_df]
for df in list_of_all_nontrending_dfs:
    df['video_id'] = df['video_id'].astype(str) 
    df['title'] = df['title'].astype(str)
    df['channel_title'] = df['channel_title'].astype(str)
    df['category_id'] = df['category_id'].astype(int)
    df['category'] = df['category'].astype(str)
    df['tags'] = df['tags'].astype(str)
    df['views'] = df['views'].astype(int)
    df['likes'] = df['likes'].astype(int)
    df['dislikes'] = df['dislikes'].astype(int)
    df['comment_count'] = df['comment_count'].astype(int)
    df['description'] = df['description'].astype(str)
full_nontrending_df = pd.concat(list_of_all_nontrending_dfs)
allDfsList = list_of_all_trending_dfs + list_of_all_nontrending_dfs + [full_trending_df] + [full_nontrending_df]
allDfsDf = pd.concat(allDfsList)
allDfsList.append(allDfsDf)

In [43]:
def insert_category_field(df):
    # add a category column using the category id from the json file
    id_to_category = {}
    with open("youtube-new/US_category_id.json", 'r') as f: # the other category json files have missing category ids. We decided to just use the US file since it contained all of them and since they are standard internationally. 
        data = json.load(f)
        for category in data['items']:
            id_to_category[category['id']] = category['snippet']['title']
        categories = []
        for id in list(df['category_id']):
            categories.append(id_to_category[str(id)])
        df.insert(4, 'category', categories)

In [44]:
## insert new category field into dataframes
for df in allDfsList:
    insert_category_field(df)

In [51]:
def find_stats(df):
    views = df['views']
    likes = df['likes']
    dislikes = df['dislikes']
    comment_count = df['comment_count']
    tags = list(df['tags'])
    tagsList = []
    for t in tags: 
        tagsList.append(len(t.split('|')))
    viewsStats = [views.sum(), views.sum() / len(views), views.min(), views.max()]
    likesStats = [likes.sum(), likes.sum() / len(likes), likes.min(), likes.max()]
    dislikesStats = [dislikes.sum(), dislikes.sum() / len(dislikes), dislikes.min(), dislikes.max()]
    commentsStats = [comment_count.sum(), comment_count.sum() / len(comment_count), comment_count.min(), comment_count.max()]
    tagsStats = [sum(tagsList), sum(tagsList) / len(tagsList), min(tagsList), max(tagsList)]
    statsDf = pd.DataFrame({'views': viewsStats, 'likes': likesStats, 'dislikes': dislikesStats, 'comment_count': commentsStats, 'tags': tagsStats}, index = ['count', 'mean', 'min', 'max']) 
    return statsDf

In [52]:
## output is a map containing all the numeric data for each df
describes = []
for df in allDfsList:
    describes.append(find_stats(df))
describesKeys = ['US_trending_df', 'CA_trending_df', 'DE_trending_df', 'FR_trending_df', 'GB_trending_df',
                'IN_trending_df', 'not_trending_us_df', 'not_trending_ca_df', 'not_trending_de_df', 'not_trending_fr_df',
                'not_trending_gb_df', 'not_trending_in_df', 
                'full_trending_df', 'full_nontrending_df', 'allDfsDf']
describeMap = dict(zip(describesKeys, describes))

In [56]:
describeMap
## find stats on a specific country: describeMap['US_trending_df']
### lets visualize this since this is all our numeric data!! 

{'US_trending_df':               views         likes      dislikes  comment_count           tags
 count  9.667177e+10  3.041147e+09  1.519782e+08   3.458882e+08  808183.000000
 mean   2.360785e+06  7.426670e+04  3.711401e+03   8.446804e+03      19.736331
 min    5.490000e+02  0.000000e+00  0.000000e+00   0.000000e+00       1.000000
 max    2.252119e+08  5.613827e+06  1.674420e+06   1.361580e+06      69.000000,
 'CA_trending_df':               views         likes      dislikes  comment_count           tags
 count  4.689198e+10  1.618180e+09  8.213792e+07   2.061618e+08  800372.000000
 mean   1.147036e+06  3.958269e+04  2.009195e+03   5.042975e+03      19.578093
 min    7.330000e+02  0.000000e+00  0.000000e+00   0.000000e+00       1.000000
 max    1.378431e+08  5.053338e+06  1.602383e+06   1.114800e+06     124.000000,
 'DE_trending_df':               views         likes      dislikes  comment_count           tags
 count  2.464512e+10  8.933955e+08  5.705903e+07   1.137744e+08  733959.000

In [57]:
###get most common tags
def get_most_common_tags(country_df):
    tags = country_df['tags'].to_string(index=False, header=False)
    split_tags = [i.replace('"', '') for i in tags.split("|")]
    stop_words = stopwords.words('english')
    filtered_tags = [word for word in split_tags if word not in stop_words]
    fdist = FreqDist(split_tags)
    most_popular_tags = fdist.most_common(1000)
    return dict(most_popular_tags)

In [58]:
## a bunch of maps containing the most popular tags for each country that are both trending and not trending and their frequency
us_trending_most_common_tags = get_most_common_tags(US_trending_df)
ca_trending_most_common_tags = get_most_common_tags(CA_trending_df)
de_trending_most_common_tags = get_most_common_tags(DE_trending_df)
fr_trending_most_common_tags = get_most_common_tags(FR_trending_df)
gb_trending_most_common_tags = get_most_common_tags(GB_trending_df)
in_trending_most_common_tags = get_most_common_tags(IN_trending_df)

us_nontrending_most_common_tags = get_most_common_tags(not_trending_us_df)
ca_nontrending_most_common_tags = get_most_common_tags(not_trending_ca_df)
de_nontrending_most_common_tags = get_most_common_tags(not_trending_de_df)
fr_nontrending_most_common_tags = get_most_common_tags(not_trending_fr_df)
gb_nontrending_most_common_tags = get_most_common_tags(not_trending_gb_df)
in_nontrending_most_common_tags = get_most_common_tags(not_trending_in_df)

### do visualization on these most common tags!!!

In [86]:
## do sentiment analysis on each of the tags
## return classifications on each of the tags in both trending and non-trending per country
positive_words_df = pd.read_fwf('positivewords.txt')
negative_words_df = pd.read_fwf('negativewords.txt')

def extract_features(words):
    return dict([(word, True) for word in words.split()])

def build_sentiment_analysis_model():
    positive_words = positive_words_df['positivewords'].values.tolist()
    negative_words = negative_words_df['negativewords'].values.tolist()
    pos_feats = [(extract_features(f), 'positive') for f in positive_words ]
    neg_feats = [(extract_features(f), 'negative') for f in negative_words ]
    dataset = pos_feats + neg_feats
    random.shuffle(dataset)
    cutoff = int(0.80 * len(dataset))
    train_data = dataset[:cutoff]
    test_data = dataset[cutoff:]

    classifier = NaiveBayesClassifier.train(train_data)
    print("Accuracy is:", classify.accuracy(classifier, test_data))
    # print(classifier.show_most_informative_features(10))
    return classifier

def execute_model(tags):
    classifications = {}
    classifier = build_sentiment_analysis_model()
    # classifier.show_most_informative_features(5)
    for tag in tags:
        classified = classifier.classify(extract_features(tag))
        classifications[tag] = classified
    return classifications

In [87]:
classifications_us_trending = execute_model(list(us_trending_most_common_tags.keys()))
classifications_ca_trending = execute_model(list(ca_trending_most_common_tags.keys()))
classifications_de_trending = execute_model(list(de_trending_most_common_tags.keys()))
classifications_fr_trending = execute_model(list(fr_trending_most_common_tags.keys()))
classifications_gb_trending = execute_model(list(gb_trending_most_common_tags.keys()))
classifications_in_trending = execute_model(list(in_trending_most_common_tags.keys()))

classifications_us_nontrending = execute_model(list(us_nontrending_most_common_tags.keys()))
classifications_ca_nontrending = execute_model(list(ca_nontrending_most_common_tags.keys()))
classifications_de_nontrending = execute_model(list(de_nontrending_most_common_tags.keys()))
classifications_fr_nontrending = execute_model(list(fr_nontrending_most_common_tags.keys()))
classifications_gb_nontrending = execute_model(list(gb_nontrending_most_common_tags.keys()))
classifications_in_nontrending = execute_model(list(in_nontrending_most_common_tags.keys()))

Accuracy is: 0.6400736422215404
Accuracy is: 0.6465173366063209
Accuracy is: 0.6351641607855171
Accuracy is: 0.6354710033752685
Accuracy is: 0.6327094200675054
Accuracy is: 0.6302546793494938
Accuracy is: 0.6311752071187481
Accuracy is: 0.6287204664007364
Accuracy is: 0.6330162626572569
Accuracy is: 0.6406873274010433
Accuracy is: 0.6413010125805462
Accuracy is: 0.6446762810678122


In [88]:
def get_sentiment_stats(classification, country):
    sentiments = list(classification.values())
    sentiments_df = pd.DataFrame(sentiments, columns=['Sentiment'])
    negatives = len(sentiments_df[sentiments_df['Sentiment'] =='negative'])
    positives = len(sentiments_df[sentiments_df['Sentiment'] =='positive'])
    total_len = len(sentiments_df)
    percentage_of_negative = negatives / total_len * 100
    percentage_of_positive = positives / total_len * 100
    ratio = positives/negatives
    ratioStr = "{} positive/negative ratio: {} -----> {}% positives of total , {}% negatives of total".format(country, ratio, percentage_of_positive, percentage_of_negative)
    print (ratioStr)

In [89]:
get_sentiment_stats(classifications_us_trending, "USA_Trending")
get_sentiment_stats(classifications_ca_trending, "Canada_Trending")
get_sentiment_stats(classifications_de_trending, "Denmark_Trending")
get_sentiment_stats(classifications_fr_trending, "France_Trending")
get_sentiment_stats(classifications_gb_trending, "GreatBritain_Trending")
get_sentiment_stats(classifications_in_trending, "India_Trending")


get_sentiment_stats(classifications_us_nontrending, "USA_NonTrending")
get_sentiment_stats(classifications_ca_nontrending, "Canada_NonTrending")
get_sentiment_stats(classifications_de_nontrending, "Denmark_NonTrending")
get_sentiment_stats(classifications_fr_nontrending, "France_NonTrending")
get_sentiment_stats(classifications_gb_nontrending, "GreatBritain_NonTrending")
get_sentiment_stats(classifications_in_nontrending, "India_NonTrending")

## visualize these!!! 

USA_Trending positive/negative ratio: 0.21951219512195122 -----> 18.0% positives of total , 82.0% negatives of total
Canada_Trending positive/negative ratio: 0.287001287001287 -----> 22.3% positives of total , 77.7% negatives of total
Denmark_Trending positive/negative ratio: 0.1848341232227488 -----> 15.6% positives of total , 84.39999999999999% negatives of total
France_Trending positive/negative ratio: 0.13765642775881684 -----> 12.1% positives of total , 87.9% negatives of total
GreatBritain_Trending positive/negative ratio: 0.24688279301745636 -----> 19.8% positives of total , 80.2% negatives of total
India_Trending positive/negative ratio: 0.3831258644536653 -----> 27.700000000000003% positives of total , 72.3% negatives of total
USA_NonTrending positive/negative ratio: 2.3333333333333335 -----> 70.0% positives of total , 30.0% negatives of total
Canada_NonTrending positive/negative ratio: 1.304147465437788 -----> 56.599999999999994% positives of total , 43.4% negatives of total


In [90]:
import spacy
spacy.load('en')
from spacy.lang.en import English
parser = English()
def tokenize(text):
    lda_tokens = []
    tokens = parser(text)
    for token in tokens:
        if token.orth_.isspace():
            continue
        elif token.like_url:
            lda_tokens.append('URL')
        elif token.orth_.startswith('@'):
            lda_tokens.append('SCREEN_NAME')
        else:
            lda_tokens.append(token.lower_)
    return lda_tokens

import nltk
nltk.download('wordnet')
from nltk.corpus import wordnet as wn
def get_lemma(word):
    lemma = wn.morphy(word)
    if lemma is None:
        return word
    else:
        return lemma
    
from nltk.stem.wordnet import WordNetLemmatizer
def get_lemma2(word):
    return WordNetLemmatizer().lemmatize(word)

nltk.download('stopwords')
en_stop = set(nltk.corpus.stopwords.words('english'))

def prepare_text_for_lda(text):
    tokens = tokenize(str(text))
    tokens = [token for token in tokens if len(token) > 4]
    tokens = [token for token in tokens if token not in en_stop]
    tokens = [get_lemma(token) for token in tokens]
    return tokens

[nltk_data] Downloading package wordnet to /Users/aravind/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/aravind/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [94]:
import random
from gensim import corpora
import gensim
import pickle
def do_LDA(lda_input):
    text_data = []
    for line in lda_input:
        tokens = prepare_text_for_lda(line)
        if random.random() > .99:
            text_data.append(tokens)
    topics, corpus, dictionary = execute_LDA(text_data)
    return topics, corpus, dictionary
    
def execute_LDA(text_data):
    dictionary = corpora.Dictionary(text_data)
    corpus = [dictionary.doc2bow(text) for text in text_data]
    pickle.dump(corpus, open('corpus.pkl', 'wb'))
    dictionary.save('dictionary.gensim')
    ldamodel = gensim.models.ldamodel.LdaModel(corpus, num_topics = 10, id2word=dictionary, passes=15)
    ldamodel.save('model5.gensim')
    topics = ldamodel.print_topics(num_words=10)
    return topics, corpus, dictionary

import re
def clean_lda_input(input):
    l = []
    for a in input:
        text = re.sub(r"http\S+", "", str(a))
        l.append(text)
    return l

In [95]:
def visualize_LDA(start, corpus, dictionary):
    if (start == True):
        lda = gensim.models.ldamodel.LdaModel.load('model5.gensim')
        lda_display = pyLDAvis.gensim.prepare(lda, corpus, dictionary, sort_topics=False)
        return lda_display

In [98]:
full_trending_lda_input = list(get_most_common_tags(full_trending_df).keys()) + list (full_trending_df.sample(11078)['description'])
topics_Full_Trending, corpus, dictionary = do_LDA(full_trending_lda_input)
lda_display = visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [99]:
for t in topics_Full_Trending:
    print(t)

(0, '0.012*"music" + 0.007*"watch?v" + 0.006*"plozaghbolese-8h8jr" + 0.005*"youtube" + 0.005*"kevin" + 0.004*"SCREEN_NAME" + 0.004*"things" + 0.004*"movie" + 0.003*"available" + 0.003*"instagram"')
(1, '0.009*"jeopardy" + 0.008*"msnbc" + 0.007*"christmas" + 0.006*"grinch" + 0.006*"ستاره" + 0.006*"official" + 0.005*"award" + 0.005*"برنامه" + 0.004*"facebook" + 0.004*"perform"')
(2, '0.008*"talent" + 0.007*"watch?v" + 0.006*"america" + 0.005*"bravo" + 0.005*"episode" + 0.004*"watch" + 0.004*"video" + 0.004*"relationship" + 0.004*"wissen" + 0.004*"fortnite"')
(3, '0.008*"subscribe" + 0.007*"video" + 0.006*"channel" + 0.006*"facebook" + 0.006*"girlfriend" + 0.006*"meghan" + 0.005*"hawaii" + 0.005*"missile" + 0.004*"youtube" + 0.004*"robot"')
(4, '0.008*"holder" + 0.008*"video" + 0.007*"family" + 0.007*"story" + 0.006*"subscribe" + 0.006*"light" + 0.005*"award" + 0.005*"latest" + 0.005*"growth" + 0.005*"bollywood"')
(5, '0.009*"interview" + 0.006*"exclusive" + 0.005*"video" + 0.005*"gopal" 

In [100]:
full_nontrending_lda_input = list(get_most_common_tags(full_nontrending_df).keys()) + list (full_nontrending_df['description'])
topics_Full_Nontrending, corpus, dictionary = do_LDA(full_nontrending_lda_input)
lda_display = visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [101]:
for t in topics_Full_Nontrending:
    print(t)

(0, '0.011*"marie" + 0.011*"madeleine" + 0.010*"toyota" + 0.006*"subscriber" + 0.006*"instagram" + 0.006*"twitter" + 0.005*"music" + 0.005*"direct" + 0.005*"facebook" + 0.005*"first"')
(1, '0.016*"watch" + 0.014*"video" + 0.010*"follow" + 0.008*"subscribe" + 0.008*"مسلسل" + 0.008*"gavin" + 0.008*"telugu" + 0.007*"michael" + 0.007*"geoff" + 0.007*"theatre"')
(2, '0.029*"subscriber" + 0.013*"jaworskyj" + 0.012*"ishare" + 0.012*"music" + 0.007*"56700" + 0.006*"video" + 0.005*"twitter" + 0.005*"fastgoodcuisine" + 0.005*"instagram" + 0.005*"randhawa"')
(3, '0.014*"twitter" + 0.011*"instagram" + 0.011*"video" + 0.010*"follow" + 0.010*"facebook" + 0.009*"music" + 0.008*"SCREEN_NAME" + 0.007*"movie" + 0.007*"subscribe" + 0.007*"mundo"')
(4, '0.006*"bella" + 0.006*"instagram" + 0.005*"twitter" + 0.005*"simulator" + 0.005*"title" + 0.005*"facebook" + 0.005*"video" + 0.004*"SCREEN_NAME" + 0.004*"hazar" + 0.004*"potato"')
(5, '0.015*"video" + 0.013*"girl" + 0.012*"comedy" + 0.011*"gujarati" + 0.00

In [121]:
allDfsDf_lda_input = list(get_most_common_tags(allDfsDf).keys()) + list (allDfsDf.sample(100000)['description'])
topics_all_dfs, corpus, dictionary = do_LDA(allDfsDf_lda_input)
lda_display = visualize_LDA(True, corpus, dictionary)
pyLDAvis.display(lda_display)

In [122]:
for t in topics_all_dfs:
    print(t)

(0, '0.005*"channel" + 0.005*"video" + 0.004*"production" + 0.003*"music" + 0.003*"world" + 0.003*"SCREEN_NAME" + 0.003*"james" + 0.003*"twitter" + 0.002*"makeup" + 0.002*"thanks"')
(1, '0.004*"video" + 0.003*"channel" + 0.003*"youtube" + 0.003*"SCREEN_NAME" + 0.003*"heart" + 0.002*"music" + 0.002*"attack" + 0.002*"facebook" + 0.002*"paddington" + 0.002*"subscribe"')
(2, '0.006*"video" + 0.005*"channel" + 0.004*"fade" + 0.004*"subscribe" + 0.004*"every" + 0.004*"twitter" + 0.004*"watch" + 0.003*"youtube" + 0.003*"voice" + 0.003*"season"')
(3, '0.007*"twitter" + 0.005*"SCREEN_NAME" + 0.005*"facebook" + 0.004*"video" + 0.004*"instagram" + 0.003*"subscribe" + 0.002*"watch" + 0.002*"drink" + 0.002*"movie" + 0.002*"2018/2019"')
(4, '0.009*"video" + 0.005*"telugu" + 0.004*"rainbow" + 0.004*"siege" + 0.003*"movie" + 0.003*"fight" + 0.003*"youtube" + 0.002*"music" + 0.002*"official" + 0.002*"trailer"')
(5, '0.016*"subscriber" + 0.005*"episode" + 0.005*"msnbc" + 0.004*"follow" + 0.004*"watch" +

In [128]:
## do following for both trending and nontrending datasets and/or per country
## do good visualizations for all!!!

## find likes to dislikes ratio ---> shows where people have the greatest divide and where people agree the most 
## which categories are the most popular? do highest average amongst likes and views ----> shows which type of videos people enjoy the most
## most trending videos in each country? do average amongst likes, views ------> shows most trending videos per country
## how long did a video stay trending? how long did it take to become trending?
## which country has the most active participation and online presence across trending and nontrending? ----> min, max, std, quartiles, counts, etc
## correlation between views, likes, dislikes, and comments
## get most frequent tags ----> run sentiment analysis on tags, run LDA model on tags
## what makes a video trending? find the minimum number/qualities required to make a video trending given our data
    ## -> for ex: need 2000 views, 1000 likes, 1500 comments, needs to be about sports, and the tags have to give off a "happy" feeling

In [138]:
not_trending_us_df.to_csv("not_trending_us_df.csv", sep='\t')
not_trending_ca_df.to_csv("not_trending_ca_df.csv", sep='\t')
not_trending_de_df.to_csv("not_trending_de_df.csv", sep='\t')
not_trending_fr_df.to_csv("not_trending_fr_df.csv", sep='\t')
not_trending_gb_df.to_csv("not_trending_gb_df.csv", sep='\t')
not_trending_in_df.to_csv("not_trending_in_df.csv", sep='\t')